## Configure training

The following os.environ lines can be customized to set the words that will be trained for, and the steps and learning rate of the training. The default values will result in the same model that is used in the micro_speech example. Run the cell to set the configuration:

In [ ]:
import os

# A comma-delimited list of the words you want to train for.
# The options are: yes,no,up,down,left,right,on,off,stop,go
# All other words will be used to train an "unknown" category.
os.environ["WANTED_WORDS"] = "zero,one,two,three,four,five,six,seven,eight,nine"

# The number of steps and learning rates can be specified as comma-separated
# lists to define the rate at each stage. For example,
# TRAINING_STEPS=15000,3000 and LEARNING_RATE=0.001,0.0001
# will run 18,000 training loops in total, with a rate of 0.001 for the first
# 15,000, and 0.0001 for the final 3,000.
os.environ["TRAINING_STEPS"]="15000,3000"
os.environ["LEARNING_RATE"]="0.001,0.0001"

# Calculate the total number of steps, which is used to identify the checkpoint
# file name.
total_steps = sum(map(lambda string: int(string),
                  os.environ["TRAINING_STEPS"].split(",")))
os.environ["TOTAL_STEPS"] = str(total_steps)

# Print the configuration to confirm it
!echo "Training these words: ${WANTED_WORDS}"
!echo "Training steps in each stage: ${TRAINING_STEPS}"
!echo "Learning rate in each stage: ${LEARNING_RATE}"
!echo "Total number of training steps: ${TOTAL_STEPS}"

## Install dependencies
Next, we'll install a GPU build of TensorFlow, so we can use GPU acceleration for training.

In [ ]:
!pip uninstall -y tensorflow-gpu tensorflow tensorflow_estimator tensorboard
!pip install -q tensorflow-estimator==1.15.1 tensorflow-gpu==1.15.2

We'll also clone the TensorFlow repository, which contains the scripts that train and freeze the model.

In [ ]:
# Clone the repository from GitHub
!git clone -q https://github.com/tensorflow/tensorflow /mnt/tmp/tensorflow
# Check out a commit that has been tested to work
# with the build of TensorFlow we're using
!git -c advice.detachedHead=false -C /mnt/tmp/tensorflow checkout 17ce384df70

## Begin training
Next, run the following script to begin training. The script will first download the training data:

In [ ]:
!python /mnt/tmp/tensorflow/tensorflow/examples/speech_commands/train.py \
--model_architecture=tiny_conv --window_stride=20 --preprocess=micro \
--wanted_words=${WANTED_WORDS} --silence_percentage=25 --unknown_percentage=25 \
--quantize=1 --verbosity=WARN --how_many_training_steps=${TRAINING_STEPS} \
--learning_rate=${LEARNING_RATE} --summaries_dir=/mnt/tmp/content/retrain_logs \
--data_dir=/mnt/tmp/content/speech_dataset --train_dir=/mnt/tmp/content/speech_commands_train \

## Training finished

Export the model in a compact format

In [ ]:
!python /mnt/tmp/tensorflow/tensorflow/examples/speech_commands/freeze.py \
--model_architecture=tiny_conv --window_stride=20 --preprocess=micro \
--wanted_words=${WANTED_WORDS} --quantize=1 --output_file=./content/tiny_conv.pb \
--start_checkpoint=/mnt/tmp/content/speech_commands_train/tiny_conv.ckpt-${TOTAL_STEPS}

## Convert the model

Run this cell to use the TensorFlow Lite converter to convert the frozen graph into the TensorFlow Lite format, fully quantized for use with embedded devices.

In [ ]:
!toco \
--graph_def_file=./content/tiny_conv.pb --output_file=./content/tiny_conv.tflite \
--input_shapes=1,49,40,1 --input_arrays=Reshape_2 --output_arrays='labels_softmax' \
--inference_type=QUANTIZED_UINT8 --mean_values=0 --std_dev_values=9.8077

Print the model size

In [ ]:
import os
model_size = os.path.getsize("./content/tiny_conv.tflite")
print("Model is %d bytes" % model_size)

Finally, we use xxd to transform the model into a source file that can be included in a C++ project and loaded by TensorFlow Lite for Microcontrollers.

In [ ]:
# Save the file as a C source file
!xxd -i ./content/tiny_conv.tflite > ./content/tiny_conv.cc
# Print the source file
!cat ./content/tiny_conv.cc